# 0 - VBM Introduction

# 1 - Code Definition
- I am leaving the code directly here so you do not need to install anything.
- Code is also exposed so you can edit as needed.

In [1]:
import os
import sys
import platform
import subprocess

class CalvinVBM:
    """
    CalvinVBM is a class for orchestrating neuroimaging data processing using Docker.
    It facilitates building Docker images and running Docker containers on individual directories containing NIfTI files.
    """

    @staticmethod
    def build_docker_image(dockerfile_path):
        """
        Builds the Docker image from the specified Dockerfile.

        Args:
            dockerfile_path (str): The path to the Dockerfile.
        """
        try:
            print("Building Docker image...")
            subprocess.call(f"docker build -t cat12:latest -f {dockerfile_path} .", shell=True, stdout=None, stderr=None)
            # subprocess.run(["docker", "build", "-t", "cat12:latest", "-f", dockerfile_path, "."], check=True)
            print("Docker image built successfully.")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while building the Docker image: {e}")
            sys.exit(1)

    @staticmethod
    def convert_path_for_docker(path):
        """
        Converts a Windows path to a Docker-compatible path (if running on Windows).

        Args:
            path (str): The original file system path.

        Returns:
            str: The converted path suitable for Docker volume mounting.
        """
        if platform.system() == "Windows":
            return path.replace("\\", "/") #.replace("C:", "C")
        return path

    @staticmethod
    def run_docker_on_folder(folder_path):
        """
        Runs the Docker container on the specified folder containing NIfTI files.

        Args:
            folder_path (str): The path to the folder containing NIfTI files.
        """
        try:
            docker_path = CalvinVBM.convert_path_for_docker(folder_path)
            print(f"Processing folder with Docker: {folder_path}")
            subprocess.run(["docker", "run", "--rm", "-v", f"{docker_path}:/data", "cat12:latest"], check=True)
            print(f"Finished processing: {folder_path}")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while processing {folder_path} with Docker: {e}")

    @staticmethod
    def submit_hpc_jobs(data_dir, docker_script_path, job_script_path):
        """
        Submits jobs to an HPC scheduler for processing each subdirectory 
        in the specified data directory.

        Args:
            data_dir (str): The path to the directory containing subdirectories with NIfTI files.
            docker_script_path (str): The path to the Docker run script.
            job_script_path (str): The path to the job submission script for HPC.
        """
        # Ensure the data directory exists
        if not os.path.isdir(data_dir):
            print(f"Data directory does not exist: {data_dir}")
            sys.exit(1)

        # Iterate over all subdirectories and submit a job for each
        for folder_name in os.listdir(data_dir):
            folder_path = os.path.join(data_dir, folder_name)
            if os.path.isdir(folder_path):
                CalvinVBM.submit_hpc_job(folder_path, docker_script_path, job_script_path)

    @staticmethod
    def submit_hpc_job(folder_path, docker_script_path, job_script_path):
        """
        Submits a single job to an HPC scheduler for processing a given folder.

        Args:
            folder_path (str): The path to the folder to process.
            docker_script_path (str): The path to the Docker run script.
            job_script_path (str): The path to the job submission script for HPC.
        """
        try:
            print(f"Submitting job for folder: {folder_path}")
            # Replace the following line with your HPC's job submission command
            subprocess.run([
                "bsub", "-J", "cat12_job", 
                "-o", "/path/to/output.txt", 
                "-e", "/path/to/error.txt", 
                "-q", "normal", 
                "-R", "rusage[mem=6000]", 
                "python", docker_script_path, folder_path
            ], check=True)
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while submitting {folder_path}: {e}")



# if __name__ == "__main__":
#     if len(sys.argv) != 3:
#         print("Usage: python calvin_vbm.py <path_to_data_directory> <path_to_dockerfile>")
#         sys.exit(1)

#     data_directory = sys.argv[1]
#     dockerfile_path = sys.argv[2]

#     CalvinVBM.submit_jobs(data_directory, dockerfile_path)



# 02 - Execution

**Option 1) Submit Jobs to Server**
- This uses LSF Job Schedler-style submission. 
- You must run this off a server which has LSF installed, such as Harvard's ERISTwo.
- Some clusters will not work well with Docker, and Singularity will be required. This is pending. 

**Option 2 - Run A Single Folder of Nifti Files**
- This does not submit to cluster. It is stable and will run on your machine. 

Define the folder with the Niftis in it

In [2]:
folder_path = r'C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\test'

Run Docker
- Estimated 15 minutes per Nifti file. 

In [3]:
CalvinVBM.build_docker_image(dockerfile_path=r"C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\containers\Dockerfile.cat12")
CalvinVBM.run_docker_on_folder(folder_path=folder_path)

Building Docker image...
Docker image built successfully.
Processing folder with Docker: C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\test
Finished processing: C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\test


**Option 3 - Multiprocess a Folder of Nifti Files**
- This is stable
- Will not provide Total Intracranial Volume. You will need to get it from reports/<file>.xml if you want to correct for Total Intracranial Volume
- This may strain your computer. However it will run locally

Define the folder with the Niftis in it

In [ ]:
folder_path = r'C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\test'

Run Docker
- Estimated 15 minutes per Nifti file.
- using 4 processors.
- Expected time to completiong = number of file / processors * 15 minutes

In [ ]:
CalvinVBM.build_docker_image(dockerfile_path=r"C:\Users\Calvin\Documents\Software\Research\nimlab\vbm\cat12\containers\Dockerfile.cat12_parallel")
CalvinVBM.run_docker_on_folder(folder_path=folder_path)

Your VBM is now running. 
 - Calvin Howard
 - e: choward12@bwh.harvard.edu